# Dataset

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

# Загрузка датасета
dataset = load_dataset("zloelias/lenta-ru")

train_data = load_dataset("zloelias/lenta-ru", split="train")

In [ ]:
import pandas as pd

# Загрузка данных
df = pd.DataFrame(train_data)

In [ ]:
print(df.info())  # Типы данных и наличие пустых значений

In [ ]:
print(df.describe(include='all'))  # Основные статистики

In [ ]:
print(df.head())

In [ ]:
import matplotlib.pyplot as plt

df['text_length'] = df['text'].apply(lambda x: len(x.split()))
plt.hist(df['text_length'], bins=50)
plt.xlabel("Длина текста (в словах)")
plt.ylabel("Количество текстов")
plt.show()

print(df['text_length'].describe())

In [ ]:
print(df['labels'].value_counts())  # Подсчёт по категориям

In [ ]:
df.drop_duplicates(subset=['text'], inplace=True)
print(f"Количество строк после удаления дубликатов: {len(df)}")

In [ ]:
# Удаление слишком коротких текстов (например, менее 10 слов)
df_balanced = df_balanced[df_balanced['text'].apply(lambda x: len(x.split()) > 10)]

In [ ]:
# Обрезка текстов до 512 слов
max_length = 512
df_balanced['text'] = df_balanced['text'].apply(lambda x: ' '.join(x.split()[:max_length]))

In [ ]:
from sklearn.utils import resample

# Создадим выборки для каждой категории
df_0 = df[df['labels'] == 0]
df_1 = df[df['labels'] == 1]
df_2 = df[df['labels'] == 2]
df_3 = df[df['labels'] == 3]
df_4 = df[df['labels'] == 4]

# Уменьшим количество данных для категории 0 до количества данных в категории 4
df_0_downsampled = resample(df_0, replace=False, n_samples=5, random_state=42)
df_1_downsampled = resample(df_1, replace=False, n_samples=5, random_state=42)
df_2_downsampled = resample(df_2, replace=False, n_samples=5, random_state=42)
df_3_downsampled = resample(df_3, replace=False, n_samples=5, random_state=42)
df_4_downsampled = resample(df_4, replace=False, n_samples=5, random_state=42)
df_balanced = pd.concat([df_0_downsampled, df_1_downsampled, df_2_downsampled, df_3_downsampled, df_4_downsampled])

print(df_balanced['labels'].value_counts())  # Проверим новое распределение

In [ ]:
# Удаляем ненужные столбцы
df_final = df_balanced.drop(columns=['title', 'topic', 'text_length', 'labels'])

# Проверяем оставшиеся столбцы
print(df_final.head(20))

In [ ]:
df_final.to_csv('final_dataset.csv', index=False)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

# Для большей наглядности будем работать с русскоязычной версией GPT от Сбера.
# Ниже находятся команды для загрузки и инициализации модели и токенизатора.
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(DEVICE)

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

train_path = 'final_dataset.csv'

# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=64)

# Создание даталодера (нарезает текст на оптимальные по длине куски)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned",       # директория с выходными данными
    overwrite_output_dir=True,      # перезаписываем содержимое выходной директории при каждом запуске
    num_train_epochs=200,           # число эпох обучения
    per_device_train_batch_size=32, # batch size для обучения
    per_device_eval_batch_size=32,  # batch size для выполнения
    warmup_steps=10,                # количество шагов для "прогрева" (управление скоростью обучения)
    gradient_accumulation_steps=16, # накопление градиента (16 шагов накапливаем градиенты для batch_size, эмуляция вычисления на пакете 16 * 32 для слабых GPU)
    )

# Инициализируем класс обучения
trainer = Trainer(
    model=model,                 # модель
    args=training_args,          # параметры обучения
    data_collator=data_collator, # загрузчик данных
    train_dataset=train_dataset, # датасет для обучения
    optimizers = (torch.optim.AdamW(model.parameters(), lr=1e-5), None) # оптимизатор
)

In [ ]:
trainer.train()

In [ ]:
texts = [
    "В Москве запущен новый проект",
    "Президент выступил с заявлением",
    "Ученые открыли новое средство",
    "Спортсмен побил рекорд",
    "Технологическая компания анонсировала",
    "Глобальные изменения климата",
    "В стране планируется провести",
    "Новый законопроект был внесен",
    "Эксперты предсказывают",
    "Пандемия привела к"
]

In [ ]:
# Генерация новостей
generated_texts = []

for text in texts:
    input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
    model.eval()
    out = model.generate(input_ids, max_length=70)
    generated_text = tokenizer.decode(out[0], skip_special_tokens=True)
    generated_texts.append(generated_text)
    print(f"Input: {text}\nGenerated News: {generated_text}\n{'-'*50}")